# ETL - PRUEBAS


In [ ]:
!pip install unidecode

from unidecode import unidecode
import pandas as pd
import string
import re
import numpy as np

# Ejecutala solamente si estás trabajando en colab
from google.colab import drive
drive.mount('/content/drive')


path = '/content/drive/Shared drives/TFM/BBDD/EXPORTED/datos_exported/'

In [ ]:
# Para poder ver todas las columnas
pd.options.display.max_columns = None

In [ ]:
def leer_y_limpiar(path):
  df = pd.read_csv('/content/drive/Shareddrives/TFM/BBDD/EXPORTED/' + path)
  df.dropna(how="all", inplace=True)      # Eliminar filas con todos los valores nulos
  df.drop_duplicates(inplace=True)        # Eliminar filas duplicadas
  df.fillna('N/A', inplace=True)          # Reemplazar valores NaN con 'N/A'
  df = df.drop(0).reset_index(drop=True)  # Establecer la primera fila como encabezados y eliminar la segunda fila

  # Define una expresión regular para buscar columnas con "none" en su nombre, las añadimos a una lista y las eliminamos de las columnas
  pattern = re.compile(r'.*none.*', flags=re.IGNORECASE)
  columns_to_remove = [col for col in df.columns if pattern.match(col)]
  df = df.drop(columns=columns_to_remove)

  return df


# Eliminar tildes y caracteres extraños de la columna ORGANIZATION_ID
def clean_organization_id(org_id):
    # Check if org_id is a string
    if isinstance(org_id, str):
        org_id = unidecode(org_id)  # Eliminar tildes
        # Eliminar caracteres extraños con una expresión regular, manteniendo los espacios
        org_id = re.sub(r'[^a-zA-Z0-9 ]', '', org_id.replace("(", "").replace(")", ""))
    # Check if org_id is a float and is not NaN
    elif isinstance(org_id, float) and not pd.isna(org_id):
        org_id = str(int(org_id))  # Convert float to int to string, removing decimal part
    else:
        org_id = ""  # For NaN or other unexpected types, convert to empty string (or handle as desired)
    return org_id

### TABLA GRANT_TYPE

In [ ]:
# 1. Cargar el DataFrame
df_grant_type = pd.read_csv('/content/drive/Shareddrives/TFM/BBDD/EXPORTED/' + 'Grant_Type_Exported_AECC_v2.csv', encoding='ISO-8859-1', delimiter=';')

# 2. Limpieza estandar
df_grant_type.dropna(how="all", inplace=True)     # Eliminar filas con todos los valores nulos
df_grant_type.drop_duplicates(inplace=True)       # Eliminar filas duplicadas
df_grant_type.fillna('N/A', inplace=True)         # Reemplazar valores NaN con 'N/A'
df_grant_type = df_grant_type.drop(0).reset_index(drop=True) # Establecer la primera fila como encabezados y eliminar la segunda fila
pattern = re.compile(r'.*none.*', flags=re.IGNORECASE)       # Define una expresión regular para buscar columnas con "none" en su nombre
columns_to_remove = [col for col in df_grant_type.columns if pattern.match(col)] # Obtiene una lista de todas las columnas que coinciden con la expresión regular
df_grant_type = df_grant_type.drop(columns=columns_to_remove)# Elimina las columnas identificadas

# 3. Convertir columnas de fecha
date_columns_grant = ["submission_start_date", "submission_end_date"]
for col in date_columns_grant:
    df_grant_type[col] = pd.to_datetime(df_grant_type[col], errors="coerce")

# 4. Limpiar columnas de texto
string_columns_grant = df_grant_type.select_dtypes(include=["object"]).columns
for col in string_columns_grant:
    df_grant_type[col] = df_grant_type[col].str.strip()

# 5. Dividir grant_description en dos columnas (español e inglés)
df_grant_type[['grant_description_en', 'grant_description_es']] = df_grant_type['grant_description'].str.split('--------------------------------------------------------------------------------', expand=True)
df_grant_type = df_grant_type.drop('grant_description', axis=1)

# 6. Corrgiendo RESOLUTION DATE
df_grant_type['resolution_date'].replace('N/A', np.nan, inplace=True)
valores_aleatorios = np.random.choice(['Q1', 'Q2', 'Q3', 'Q4'], size=len(df_grant_type))
df_grant_type['resolution_date'] = df_grant_type['resolution_date'].fillna(pd.Series(valores_aleatorios, index=df_grant_type.index))

#    Patrón de expresión regular para encontrar nombres de trimestres en varios idiomas
patron_trimestre = re.compile(r'(Primer|Primero|First|1er|1st|Segundo|Second|2do|2nd|Tercer|Tercero|Third|3er|3rd|Cuarto|Cuarto|Fourth|4to|4th)\s', re.IGNORECASE)

#    Función para reemplazar nombres de trimestres con Q1, Q2, etc.
def reemplazar_nombre_trimestre(match):
    trimestre = match.group(1)

    if trimestre:
        if "Primer" in trimestre or "First" in trimestre or "1er" in trimestre or "1st" in trimestre:
            return 'Q1'
        elif "Segundo" in trimestre or "Second" in trimestre or "2do" in trimestre or "2nd" in trimestre:
            return 'Q2'
        elif "Tercer" in trimestre or "Third" in trimestre or "3er" in trimestre or "3rd" in trimestre:
            return 'Q3'
        elif "Cuarto" in trimestre or "Fourth" in trimestre or "4to" in trimestre or "4th" in trimestre:
            return 'Q4'
    return match.group(0)

df_grant_type['resolution_date'] = df_grant_type['resolution_date'].str.replace(patron_trimestre, reemplazar_nombre_trimestre)

#    Extraer los dos primeros caracteres de 'resolution_date'
df_grant_type['resolution_date'] = df_grant_type['resolution_date'].str.slice(0, 2)

# 7. Añadimos un registro por defecto
nuevo_registro = pd.Series({'grant_type': 'default'}, name=0)
df_grant_type = df_grant_type.append(nuevo_registro)

# 8. Convertir Tipos
df_grant_type['grant_year'].fillna('0', inplace=True)
df_grant_type['grant_year'] = df_grant_type['grant_year'].astype(int)
df_grant_type['grant_duration'] = df_grant_type['grant_duration'].str.replace(',', '.').astype(float)
df_grant_type['grant_amount'] = df_grant_type['grant_amount'].str.replace(',', '.').astype(float)

In [ ]:
df_grant_type.head(2)

,grant_type,grant_status,submission_start_date,submission_end_date,resolution_date,grant_year,grant_eoi,grant_classification,grant_subclassification,grant_recipient,grant_duration,grant_amount,grant_description_en,grant_description_es
0,AECC IMPACTO 2023,Closed,2023-09-05,2023-06-16,Q4,2023,No,Innovación,Impact,Start-ups,5.0,450000.0,AECC IMPACTO es un instrumento financiero en f...,None
1,AECC Talent 2024,Draft,2023-09-28,2023-11-23,Q3,2024,No,Talento,Talent,Posdoc <8 year research experience,3.0,205200.0,The MSCA COFUND postdoctoral fellowship progra...,None


In [ ]:
df_grant_type.dtypes

grant_type                         object
grant_status                       object
submission_start_date      datetime64[ns]
submission_end_date        datetime64[ns]
resolution_date                    object
grant_year                          int64
grant_eoi                          object
grant_classification               object
grant_subclassification            object
grant_recipient                    object
grant_duration                    float64
grant_amount                      float64
grant_description_en               object
grant_description_es               object
dtype: object

### TABLA INVOICE

In [ ]:
# 1. Leer el archivo Excel en un DataFrame.
#    FRIENDLY REMINDER:Especificar low_memory=False si se enfrenta a una advertencia Dtype
df_invoice = leer_y_limpiar("Invoice_Exported_AECC_v3.csv")

# 2. Convertir columnas a tipos de datos apropiados
#    Convertir campos de fecha
date_columns_invoice = ["invoice_date", "payment_date", "starting_evaluation_period", "end_evaluation_period"]
for col in date_columns_invoice:
    df_invoice[col] = pd.to_datetime(df_invoice[col], errors="coerce")
#    Convertir la columna invoice_amount  a tipo float
df_invoice['invoice_amount'] = pd.to_numeric(df_invoice['invoice_amount'], errors='coerce')

# 3. Limpiar cadenas de texto
string_columns_invoice = df_invoice.select_dtypes(include=["object"]).columns
for col in string_columns_invoice:
    df_invoice[col] = df_invoice[col].str.strip()

# 4. Supplier_id
def find_shortest_supplier_id(group):
    # Elimina los caracteres de puntuación de todos los supplier_id en el grupo
    cleaned_group = group.str.translate(str.maketrans('', '', string.punctuation))
    # Encuentra el supplier_id más corto entre los cleaned
    shortest = cleaned_group.loc[cleaned_group.str.len().idxmin()]
    return shortest

# Agrupar por supplier_vat y aplicar la función a cada grupo
shortest_supplier_id = df_invoice.groupby('supplier_vat')['supplier_id'].transform(find_shortest_supplier_id)

# Asignar los valores de supplier_id más cortos a la columna supplier_id
df_invoice['supplier_id'] = shortest_supplier_id

# 5. Convertir la columna invoice_amount  a tipo float
df_invoice['invoice_amount'] = pd.to_numeric(df_invoice['invoice_amount'], errors='coerce')

#6. Aplicar la función de limpieza a la columna 'organization_id'
df_invoice['organization_id'] = df_invoice['organization_id'].apply(clean_organization_id)

# Máscara booleana para identificar las filas que contienen "la Rioja"
mask = df_invoice['organization_id'].str.contains("la Rioja", case=False, na=False)

# Aplicar el cambio solo a las filas identificadas por la máscara
df_invoice.loc[mask, 'organization_id'] = df_invoice.loc[mask, 'organization_id'].str.replace("la Rioja", "La Rioja", case=False)

In [ ]:
df_invoice.head(2)

,project_id,invoice_name,invoice_date,payment_date,reporting_year,starting_evaluation_period,end_evaluation_period,supplier_id,supplier_vat,concept,subconcept,concept_description,organization_claim,organization_id,organization_economic_justification,organization_vat,invoice_amount,invoice_manager,user_name,expense_type
0,INNO20011MART,22F0199S00001768,2022-10-05,2022-05-18,Y2,2021-01-06,2022-05-31,AVORIS RETAIL DIVISION SL,B07012107,TRAVEL EXPENSES,Meeting registration,AVORIS RETAIL DIVISION S.L.,IDIBELL,Instituto de Investigacion Biomedica de Bellvi...,Instituto de Investigación Biomédica de Bellvi...,G58863317,175.29,Gabriel Capella,OSCAR MARTINEZ,External
1,INNO20011MART,22F0199S00001872,2022-12-05,2022-05-24,Y2,2021-01-06,2022-05-31,AVORIS RETAIL DIVISION SL,B07012107,EXPENDABLE MATERIAL,Expendable laboratory material,AVORIS RETAIL DIVISION S.L.,IDIBELL,Instituto de Investigacion Biomedica de Bellvi...,Instituto de Investigación Biomédica de Bellvi...,G58863317,45.82,Gabriel Capella,OSCAR MARTINEZ,External


In [ ]:
df_invoice.dtypes

project_id                                     object
invoice_name                                   object
invoice_date                           datetime64[ns]
payment_date                           datetime64[ns]
reporting_year                                 object
starting_evaluation_period             datetime64[ns]
end_evaluation_period                  datetime64[ns]
supplier_id                                    object
supplier_vat                                   object
concept                                        object
subconcept                                     object
concept_description                            object
organization_claim                             object
organization_id                                object
organization_economic_justification            object
organization_vat                               object
invoice_amount                                float64
invoice_manager                                object
user_name                   

### TABLE ORGANIZATIONS


In [ ]:
# Leer el archivo Excel en un DataFrame
df_organizations = leer_y_limpiar("Organizations_Exported_AECC_v2_treated.csv")

# 1. Reemplazar valores no convertibles con 0
df_organizations['organization_postcode'] = pd.to_numeric(df_organizations['organization_postcode'], errors='coerce').fillna(0)
df_organizations['organization_fiscal_postcode'] = pd.to_numeric(df_organizations['organization_fiscal_postcode'], errors='coerce').fillna(0)

# 2. Convertir la columna a tipo entero
df_organizations['organization_postcode'] = df_organizations['organization_postcode'].astype(int)
df_organizations['organization_fiscal_postcode'] = df_organizations['organization_fiscal_postcode'].astype(int)

# 3. Limpiar cadenas de texto
string_columns_organizations = df_organizations.select_dtypes(include=["object"]).columns
for col in string_columns_organizations:
    df_organizations[col] = df_organizations[col].str.strip()

# 4. Eliminar tildes y caracteres extraños de la columna ORGANIZATION_ID
df_organizations['organization_id'] = df_organizations['organization_id'].apply(clean_organization_id)

# 5. Identificar la fila y columna específica y reemplazar "la" por "La"
df_organizations.loc[388, 'organization_id'] = df_organizations.loc[388, 'organization_id'].replace(" la ", " La ")

default_values = {
    'organization_id': 'default',
    'organization_legal_name': 'N/A',
    'organization_vat': 'N/A',
    'organization_address': 'N/A',
    'organization_fiscal_address': 'N/A',
    'organization_city': 'N/A',
    'organization_fiscal_city': 'N/A',
    'organization_province': 'N/A',
    'organization_fiscal_province': 'N/A',
    'organization_community_aut': 'N/A',
    'organization_country': 'N/A',
    'organization_postcode': 0,
    'organization_fiscal_postcode': 0,
    'organization_primary_contact': 'N/A',
    'organization_parent_company': 'N/A',
    'organization_type': 'N/A',
    'organization_status': 'N/A',
    'organization_legal_status': 'N/A'
}

# Insertar la fila en el DataFrame original en la parte superior
df_organizations = df_organizations.append(default_values, ignore_index=True)

In [ ]:
df_organizations.head(2)

,organization_id,organization_legal_name,organization_vat,organization_address,organization_fiscal_address,organization_city,organization_fiscal_city,organization_province,organization_fiscal_province,organization_community_aut,organization_country,organization_postcode,organization_fiscal_postcode,organization_primary_contact,organization_parent_company,organization_type,organization_status,organization_legal_status
0,Achucarro Basque Center for Neuroscience,FUNDACION ACHUCARRO BASQUE CENTER FOR NEUROSCI...,G95686051,Parque Tecnológico de Bizkaia,Parque Tecnológico de Bizkaia,N/A,Vizcaya,Vizcaya,Vizcaya,País Vasco,Spain,0,48170,N/A,Research Centers,Research Center,Pending Validation,N/A
1,Agencia Aragonesa para la Investigacion y el D...,FUNDACION AGENCIA ARAGONESA PARA LA INVESTIGAC...,G99085797,"C María de Luna,11","C María de Luna,11",N/A,Zaragoza,Zaragoza,Zaragoza,Aragón,Spain,0,50018,N/A,Research Centers,Research Center,Grant Research Center,N/A


In [ ]:
df_organizations.dtypes

organization_id                 object
organization_legal_name         object
organization_vat                object
organization_address            object
organization_fiscal_address     object
organization_city               object
organization_fiscal_city        object
organization_province           object
organization_fiscal_province    object
organization_community_aut      object
organization_country            object
organization_postcode            int64
organization_fiscal_postcode     int64
organization_primary_contact    object
organization_parent_company     object
organization_type               object
organization_status             object
organization_legal_status       object
dtype: object

### TABLE PROJECTS



In [ ]:
# 1. Leer el archivo Projects_Exported_AECC_v3.xlsx
df_projects = leer_y_limpiar("Projects_Exported_AECC_v3.csv")

# 2. Convertir columnas de fecha
date_columns_grant = ["project_start_date", "project_end_date", "project_updated_end_date", "project_close_date"]
for col in date_columns_grant:
    df_projects[col] = pd.to_datetime(df_projects[col], errors='coerce')

# 3. Eliminar simbolo del euro y convertir a float
money_columns = ["project_initial_assigned_amount", "project_current_assigned_amount", "project_paid_amount", "project_total_extensions_amount"]
for col in money_columns:
    df_projects[col] = pd.to_numeric(df_projects[col].replace('[\€,]', '', regex=True), errors='coerce')

# 4. Corrigiendo columnas
df_projects = df_projects.drop(columns=['research_team_table'])
df_projects.rename(columns={'type_cancer': 'cancer_code'}, inplace=True)
df_projects.rename(columns={'phase_cancer': 'cancer_phase_code'}, inplace=True)
df_projects.rename(columns={'research_type': 'research_type_code'}, inplace=True)
df_projects = df_projects.drop(columns=['protectable_ip'])
df_projects['project_status'] = df_projects['project_status'].replace(r'\n', '', regex=True)

# 5. Extrayendo el año de columnna granted_year
def extract_year(value):
    if "año" in value:
        return value.split(" ")[-1]  # Esto asume que el año siempre es la última palabra
    return None

#    Extraer el año de la columna 'granted_year'
df_projects['granted_year'] = df_projects['granted_year'].apply(lambda x: extract_year(x) if x else None)

#    Para las celdas vacías en 'granted_year', extraer el año de 'project_start_date'
df_projects['granted_year'] = df_projects.apply(
    lambda row: str(row['project_start_date'].year) if pd.isnull(row['granted_year']) else row['granted_year'],
    axis=1
)
#   Fin de las modificacionesde columna granted_year

# 8. Eliminar tildes y caracteres extraños
df_projects['organization_id'] = df_projects['organization_id'].apply(clean_organization_id)
df_projects['organization_id'] = df_projects['organization_id'].replace('Universidad de la Rioja UniRIOJA', 'Universidad de La Rioja UniRIOJA', regex=True)

# 9. Eliminar duplicados
df_projects = df_projects.drop_duplicates(subset=['project_id'])

In [ ]:
df_projects.head(2)

,project_id,grant_type,project_status,project_phase,user_name,user_dni,organization_id,project_start_date,project_end_date,project_updated_end_date,project_close_date,project_initial_assigned_amount,project_current_assigned_amount,project_paid_amount,project_total_extensions_amount,organization_province,organization_community_aut,research_type_code,research_type_innova,research_type_startup,project_name,abstract,cancer_code,cancer_phase_code,aei_evaluation_score,sponsor_name,granted_year,keywords
0,GEACC19001,Grupos Acceleratores 2019,Approved,Prórroga,Ignacio Javier Melero Bermejo,05390884Y,Cancer Center Clinica Universidad de Navarra C...,2019-01-12,2022-11-30,2024-11-30,NaT,1080217.39,1643015.00,1393920.81,562797.61,Navarra,Comunidad Foral de Navarra,"Pre-Clinical, Clinical",N/A,N/A,Local immunoradiotherapy (LIRT)\n,This project aims to test intratumour injectio...,Breast Cancer,Treatment,N/A,N/A,2019,N/A
1,GEACC18002SANM,Grupos Acceleratores 2018,Approved,Prórroga,Jesús San Miguel Izquierdo,13059038Y,Cancer Center Clinica Universidad de Navarra C...,2018-01-12,2021-11-30,2023-11-30,NaT,1283632.58,1844263.76,1677743.98,560631.18,Navarra,Comunidad Foral de Navarra,"Pre-Clinical, Clinical",N/A,N/A,Early detection and intervention: Understandin...,Although great strides have been made in the m...,Hematological Malignancies,"Early Diagnosis, Resistance",86,Iberdrola,2018,N/A


In [ ]:
df_projects.dtypes

project_id                                 object
grant_type                                 object
project_status                             object
project_phase                              object
user_name                                  object
user_dni                                   object
organization_id                            object
project_start_date                 datetime64[ns]
project_end_date                   datetime64[ns]
project_updated_end_date           datetime64[ns]
project_close_date                 datetime64[ns]
project_initial_assigned_amount           float64
project_current_assigned_amount           float64
project_paid_amount                       float64
project_total_extensions_amount           float64
organization_province                      object
organization_community_aut                 object
research_type_code                         object
research_type_innova                       object
research_type_startup                      object


In [ ]:
# @title TABLA AUX NEO4J: Cancer Types
df_fusion_ct_1 = pd.DataFrame()
df_fusion_ct_2 = pd.DataFrame()
df_fusion_rtc_3 = pd.DataFrame()

# 1. Divide la columna 'Tipos de Cancer (Varios)' en columnas separadas para cada tipo de cáncer
df_fusion_ct_1 = df_projects[['project_id','cancer_code', 'research_type_code']]

# 2. Limpiando la columnas
df_fusion_ct_1 = df_fusion_ct_1.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df_fusion_ct_1['cancer_code'] = df_fusion_ct_1['cancer_code'].str.split(',')
df_fusion_ct_1['cancer_code'] = df_fusion_ct_1['cancer_code'].apply(lambda x: [element.strip() for element in x])
df_fusion_ct_1['cancer_code'] = df_fusion_ct_1['cancer_code'].apply(lambda x: sorted([element.replace(" ", "_") for element in x]))
df_fusion_ct_1['cancer_code'] = df_fusion_ct_1['cancer_code'].apply(lambda x: [element.replace("acute_mieloblastic_leukemia", "acute_myeloblastic_leukemia") for element in x])
df_fusion_ct_1['cancer_code'] = df_fusion_ct_1['cancer_code'].apply(lambda x: [element.replace("acute_lymphoblastic_leukemia_(all)", "acute_myeloblastic_leukemia") for element in x])
df_fusion_ct_1['cancer_code'] = df_fusion_ct_1['cancer_code'].apply(lambda x: [element.replace("nasopharyngeal_carcinoma_/_cavum", "nasopharyngeal_carcinoma") for element in x])
df_fusion_ct_1['cancer_code'] = df_fusion_ct_1['cancer_code'].apply(lambda x: [element.replace("central_nervous_systmen", "central_nervous_system") for element in x])
df_fusion_ct_1 = df_fusion_ct_1.dropna(subset=['cancer_code'])

df_fusion_ct_1['research_type_code'] = df_fusion_ct_1['research_type_code'].str.split(',')
df_fusion_ct_1['research_type_code'] = df_fusion_ct_1['research_type_code'].apply(lambda x: [element.strip() for element in x])
df_fusion_ct_1['research_type_code'] = df_fusion_ct_1['research_type_code'].apply(lambda x: sorted([element.replace(" ", "_") for element in x]))

df_fusion_ct_2['cancer_code'] = df_fusion_ct_1['cancer_code']
df_fusion_ct_2.drop_duplicates(inplace=True)

df_fusion_rtc_3['research_type_code'] = df_fusion_ct_1['research_type_code']
df_fusion_rtc_3 = df_fusion_rtc_3.explode('research_type_code')
df_fusion_rtc_3.drop_duplicates(inplace=True)

In [ ]:
df_fusion_ct_2.head(1)

,cancer_code
0,[breast_cancer]


In [ ]:
df_fusion_ct_1.head(1)

,project_id,cancer_code,research_type_code
0,geacc19001,[breast_cancer],"[clinical, pre-clinical]"


In [ ]:
df_fusion_rtc_3.head(1)

,research_type_code
0,clinical


####  RESEARCH TYPE

In [ ]:
# 1. Crear un DataFrame vacío
df_research_types = pd.DataFrame()

# 2. Divide la columna 'Tipos de Research (Varios)' en columnas separadas para cada tipo de cáncer
df_research_types = df_projects['research_type_code'].str.get_dummies(', ')

# 3. Elimina los espacios en blanco en las etiquetas de las columnas
df_research_types.columns = df_research_types.columns.str.strip()
df_research_types.columns = df_research_types.columns.str.replace('-', '_').str.lower()
df_research_types['psycho_oncological'] = df_research_types['psycho_oncological'] + df_research_types['psyco_oncological']
df_research_types = df_research_types.drop(columns=['psyco_oncological'])

# 4. Limpieza 1 (mantener este orden)
df_research_types = df_research_types.drop(columns=['n/a'])

# 5. Agregar una nueva columna 'research_code' que contiene la concatenación de los valores de cada fila en las columnas existentes
df_research_types['research_type_code'] = df_research_types.apply(lambda row: ''.join(map(str, row)), axis=1)

# 6. Reemplazar los valores en la columna de df_projects por los valores de 'research_code' de df_research_types
df_projects['research_type_code'] = df_research_types['research_type_code'].values

# 7. Limpieza 2 (mantener este orden)
df_research_types = df_research_types.drop_duplicates(subset=['research_type_code']) # Eliminar filas duplicadas en el DataFrame

# 8. Mueve la última columna a la primera posición
last_column = df_research_types.pop(df_research_types.columns[-1])
df_research_types.insert(0, last_column.name, last_column)

# 9. Creamos una opcion por defecto
df_research_types.rename(columns={'others': 'other_types'}, inplace=True)

In [ ]:
df_research_types.head(2)

,research_type_code,basic,clinical,epidemiology,innovative,other_types,pre_clinical,psycho_oncological,translational
0,01000100,0,1,0,0,0,1,0,0
2,01000101,0,1,0,0,0,1,0,1


####  CANCER TYPES

In [ ]:
# 1. Crear un DataFrame vacío
df_cancer_types = pd.DataFrame()

# 2. Divide la columna 'Tipos de Cancer (Varios)' en columnas separadas para cada tipo de cáncer
df_cancer_types = df_projects['cancer_code'].str.get_dummies(', ')

# 3. Otros cambios
df_cancer_types.columns = df_cancer_types.columns.str.replace('-', '_').str.lower()
df_cancer_types = df_cancer_types.drop(columns=['n/a'])
df_cancer_types['central nervous system'] = df_cancer_types['central nervous system'] + df_cancer_types['central nervous systmen']
df_cancer_types = df_cancer_types.drop(columns=['central nervous systmen'])
df_cancer_types.columns = df_cancer_types.columns.str.replace(' ', '_').str.lower()
df_cancer_types['acute_myeloblastic_leukemia'] = df_cancer_types['acute_mieloblastic_leukemia'] + df_cancer_types['acute_myeloblastic_leukemia']
df_cancer_types = df_cancer_types.drop(columns=['acute_mieloblastic_leukemia'])
df_cancer_types.rename(columns={'acute_lymphoblastic_leukemia_(all)': 'acute_lymphoblastic_leukemia'}, inplace=True)
df_cancer_types.rename(columns={'nasopharyngeal_carcinoma_/_cavum': 'nasopharyngeal_carcinoma'}, inplace=True)

# 4. Agregar una nueva columna 'cancer_code' que contiene la concatenación de los valores de cada fila en las columnas existentes
df_cancer_types['cancer_code'] = df_cancer_types.apply(lambda row: ''.join(map(str, row)), axis=1)

# 5. Reemplazar los valores en la columna 'cancer_code' de df_projects por los valores de 'cancer_code' de df_cancer_types
df_projects['cancer_code'] = df_cancer_types['cancer_code'].values

# 6. Eliminar filas duplicadas en el DataFrame
df_cancer_types = df_cancer_types.drop_duplicates(subset=['cancer_code'])

# 7. Mueve la última columna a la primera posición
last_column = df_cancer_types.pop(df_cancer_types.columns[-1])
df_cancer_types.insert(0, last_column.name, last_column)
df_cancer_types = df_cancer_types.iloc[:, :-1]

# 8. Creamos una opcion por defecto
df_cancer_types = df_cancer_types.rename(columns={"others": "other_types"})

In [ ]:
df_cancer_types.head(2)

,cancer_code,acute_lymphoblastic_leukemia,acute_myeloblastic_leukemia,anal_cancer,bile_duct_cancer,bladder_cancer,breast_cancer,cancer_in_children,central_nervous_system,cervix_cancer,childhood_cancer,chronic_myelogenous_leukemia,colon_cancer,colorectal_cancer,cronic_limphatic_leukemia,esophagus_cancer,gallblader_cancer,hematological_malignancies,hodgkin_lymphoma,kidney_cancer,laringeal_cancer,linfomas_no_hodgkin,liver_cancer,lung_cancer,melanoma,melanoma_skin_cancer,mesothelioma,multiple_myeloma,nasopharyngeal_carcinoma,non_hodgkin_lymphoma,osteosarcoma,other_types,ovarian_cancer,pancreatic_cancer,prostate_cancer,rare_type_of__cancer,rectal_cancer,skin_carcinoma,small_intestine_cancer,soft_tissue_sarcoma,stomach_cancer,testicular_cancer,thyroid_cancer,uterine_and_endometrial_cancer,vaginal_cancer
0,000001000000000000000000000000000000000000000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,000000000000000010000000000000000000000000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


####   CANCER PHASES

In [ ]:
# 1. Crear un DataFrame vacío
df_cancer_phases = pd.DataFrame()

# 2. Divide la columna 'Tipos de Cáncer (Varios)' en columnas separadas para cada tipo de cáncer
df_cancer_phases = df_projects['cancer_phase_code'].str.get_dummies(', ')

df_cancer_phases.columns = df_cancer_phases.columns.str.replace('-', '_').str.lower()

column_name = 'n/a'  # Nombre de la columna que quieres eliminar
if column_name in df_cancer_phases.columns:
    df_cancer_phases = df_cancer_phases.drop(columns=[column_name])

# 3. Otros cambios
last_column = df_cancer_phases.pop(df_cancer_phases.columns[-1]) # Mueve la última columna a la primera posición
df_cancer_phases.insert(0, last_column.name, last_column)
df_cancer_phases.rename(columns={'early diagnosis': 'early_diagnosis'}, inplace=True)
df_cancer_phases['classification_diagnosis_final'] = df_cancer_phases['clasiffication and diagnosis'] + df_cancer_phases['classification and diagnosis']
df_cancer_phases = df_cancer_phases.drop(columns=['clasiffication and diagnosis'])
df_cancer_phases = df_cancer_phases.drop(columns=['classification and diagnosis'])
df_cancer_phases.columns = df_cancer_phases.columns.str.replace(' ', '_').str.lower()
df_cancer_phases = df_cancer_phases.rename(columns={'classification_diagnosis_final': 'classification_diagnosis'})
df_cancer_phases = df_cancer_phases.rename(columns={'others': 'other_types'})

# 4. Agregar una nueva columna 'cancer_phase_code' que contiene la concatenación de los valores de cada fila en las columnas existentes
df_cancer_phases['cancer_phase_code'] = df_cancer_phases.apply(lambda row: ''.join(map(str, row)), axis=1)
# 5. Reemplazar los valores en la columna 'phase_cancer' de df_projects por los valores de 'cancer_phase_code' de df_cancer_phases
df_projects['cancer_phase_code'] = df_cancer_phases['cancer_phase_code'].values
# 6. Eliminar filas duplicadas en el DataFrame
df_cancer_phases = df_cancer_phases.drop_duplicates(subset=['cancer_phase_code'])
# 7. Mueve la última columna a la primera posición
last_column = df_cancer_phases.pop(df_cancer_phases.columns[-1])
df_cancer_phases.insert(0, last_column.name, last_column)

In [ ]:
df_cancer_phases.head(2)

,cancer_phase_code,treatment,early_diagnosis,metastasis,other_types,prevention,resistance,classification_diagnosis
0,1000000,1,0,0,0,0,0,0
1,0100010,0,1,0,0,0,1,0


In [ ]:
# @title inter PROJECT_RESEARCH_TYPE
df_project_research_type = df_projects[['project_id', 'research_type_code']]
df_project_research_type = df_project_research_type.drop_duplicates()
df_project_research_type.head(2)

,project_id,research_type_code
0,GEACC19001,01000100
1,GEACC18002SANM,01000100


In [ ]:
# @title inter PROJECT_RESEARCH_TYPE
df_project_cancer_type = df_projects[['project_id', 'cancer_code']]
df_project_cancer_type = df_project_cancer_type.drop_duplicates()
df_project_cancer_type.head(2)

,project_id,cancer_code
0,GEACC19001,000001000000000000000000000000000000000000000
1,GEACC18002SANM,000000000000000010000000000000000000000000000


In [ ]:
# @title inter PROJECT_CANCER_PHASES
df_project_cancer_phases = df_projects[['project_id', 'cancer_phase_code']]
df_project_cancer_phases = df_project_cancer_phases.drop_duplicates()
df_project_cancer_phases.head(2)

,project_id,cancer_phase_code
0,GEACC19001,1000000
1,GEACC18002SANM,0100010


In [ ]:
# @title TABLA PROJECTS FINAL
valid_project_ids = set(df_grant_type['grant_type'])
df_projects.loc[~df_projects['grant_type'].isin(valid_project_ids), 'grant_type'] = 'default'
df_projects.head(2)

,project_id,grant_type,project_status,project_phase,user_name,user_dni,organization_id,project_start_date,project_end_date,project_updated_end_date,project_close_date,project_initial_assigned_amount,project_current_assigned_amount,project_paid_amount,project_total_extensions_amount,organization_province,organization_community_aut,research_type_code,research_type_innova,research_type_startup,project_name,abstract,cancer_code,cancer_phase_code,aei_evaluation_score,sponsor_name,granted_year,keywords
0,GEACC19001,Grupos Acceleratores 2019,Approved,Prórroga,Ignacio Javier Melero Bermejo,05390884Y,Cancer Center Clinica Universidad de Navarra C...,2019-01-12,2022-11-30,2024-11-30,NaT,1080217.39,1643015.00,1393920.81,562797.61,Navarra,Comunidad Foral de Navarra,01000100,N/A,N/A,Local immunoradiotherapy (LIRT)\n,This project aims to test intratumour injectio...,000001000000000000000000000000000000000000000,1000000,N/A,N/A,2019,N/A
1,GEACC18002SANM,Grupos Acceleratores 2018,Approved,Prórroga,Jesús San Miguel Izquierdo,13059038Y,Cancer Center Clinica Universidad de Navarra C...,2018-01-12,2021-11-30,2023-11-30,NaT,1283632.58,1844263.76,1677743.98,560631.18,Navarra,Comunidad Foral de Navarra,01000100,N/A,N/A,Early detection and intervention: Understandin...,Although great strides have been made in the m...,000000000000000010000000000000000000000000000,0100010,86,Iberdrola,2018,N/A


#### TABLE RESEARCH TEAM

In [ ]:
# 1. Crear un DataFrame vacío
#    FRIENDLY REMINDER: El csv del drive tiene encabezado dif al de OneDrive, y la columna Status ya no está
df_research_team = pd.read_csv('/content/drive/Shareddrives/TFM/BBDD/EXPORTED/Research_Team_Exported_v1.csv', skiprows=1)

first_id = df_research_team.columns[0]    # Guardar el primer ID de proyecto que está en el encabezado
df_research_team.columns.values[0] = 'ID' # Cambiar el nombre de la columna a 'ID' antes de la propagación
df_research_team.loc[0, 'ID'] = first_id  # Insertar el primer ID de proyecto en la primera fila de la columna 'ID'
df_research_team['ID'] = df_research_team['ID'].fillna(method='ffill') # Propagar el ID del proyecto hacia abajo

# 2. Crear una máscaras
mask_surname = df_research_team['First surname'].str.contains('Surname/s', na=False)
mask_first_surname = df_research_team['First surname'].str.contains('First surname', na=False)

# 3. Funcion delicada, suerte soldados
df_research_team['group'] = None   # Inicializar una columna para ayudar a identificar los grupos de filas que deben ser desplazadas juntas
group_counter = 0                  # Inicializar un contador para los grupos

#    Iterar a través de cada fila en el DataFrame
for i, row in df_research_team.iterrows():
    if mask_surname[i]:           # Si la fila contiene 'Surname/s', incrementar el contador de grupo y asignar el número de grupo a la columna 'group'
        group_counter += 1
    elif mask_first_surname[i]:   # Si la fila contiene 'First surname', resetear el contador de grupo a 0
        group_counter = 0

    df_research_team.at[i, 'group'] = group_counter # Asignar el número de grupo a la columna 'group'

# 4. Filtrar el DataFrame para obtener solo las filas que necesitan ser desplazadas && Realizar el desplazamiento en estas filas
df_to_shift = df_research_team[df_research_team['group'] > 0]
df_research_team.loc[df_research_team['group'] > 0, 'Second Surname':] = df_to_shift.loc[:, 'Second Surname':].shift(periods=1, axis=1)

# 5. Eliminar las filas donde 'Name' es NaN && 'Name' contiene 'Additional Team Members - Submitted by Applicant'
df_research_team = df_research_team.dropna(subset=['Name'])
df_research_team = df_research_team[~df_research_team['Name'].str.contains('Additional Team Members - Submitted by Applicant', na=False)]

# 6. Eliminar las filas donde 'First surname' contiene 'Surname/s' o donde 'First surname' contiene 'First surname'
df_research_team = df_research_team[~df_research_team['First surname'].str.contains('Surname/s', na=False)]
df_research_team = df_research_team[~df_research_team['First surname'].str.contains('First surname', na=False)]

# 7. Elimina la última columna y la tercera
df_research_team = df_research_team.drop(columns=[df_research_team.columns[3], df_research_team.columns[-1]])

# 8. Renombrando columnas
df_research_team = df_research_team.rename(columns={
    'ID': 'project_id',
    'Name': 'nombre',
    'First surname': 'apellido',
    'Email': 'email',
    'ID type': 'user_id_type',
    'ID number': 'user_dni',
    'Center name': 'organization_id',
    'Position': 'job_position'
})

# 9. Máscara 1 booleana para identificar las filas que contienen "VHIO" && Aplicar el cambio solo a las filas identificadas por la máscara
mask = df_research_team['organization_id'].str.contains("VHIO", case=False, na=False)
df_research_team.loc[mask, 'organization_id'] = df_research_team.loc[mask, 'organization_id'].str.replace("VHIO", "Vall dHebron Instituto de Oncologia VHIO", case=False)

# 10. Máscara 2 booleana para identificar las filas que contienen "CBMSO" && # Aplicar el cambio solo a las filas identificadas por la máscara
mask = df_research_team['organization_id'].str.contains("CBMSO", case=False, na=False)
df_research_team.loc[mask, 'organization_id'] = df_research_team.loc[mask, 'organization_id'].str.replace("CBMSO", "Centro de Biologia Molecular Severo Ochoa CBM", case=False)

# 11. Máscara 3 booleana para identificar las filas que contienen "CNIC" && Aplicar el cambio solo a las filas identificadas por la máscara
mask = df_research_team['organization_id'].str.contains("CNIC", case=False, na=False)
df_research_team.loc[mask, 'organization_id'] = df_research_team.loc[mask, 'organization_id'].str.replace("CNIC", "Centro Nacional de Investigaciones Cardiovasculares CNIC", case=False)

# 12. Aplicar la función de limpieza a la columna 'organization_id'
df_research_team['organization_id'] = df_research_team['organization_id'].apply(clean_organization_id)

# 13. Reemplazar los valores NaN por "default" en la columna "Center name"
df_research_team['organization_id'].fillna('default', inplace=True)

# 14. Asegúrate de que todos los organization_id en df_research_team existen en df_organizations
valid_organization_ids = set(df_organizations['organization_id'])
df_research_team = df_research_team[df_research_team['organization_id'].isin(valid_organization_ids)]

# 15. Asegúrate de que todos los project_id en df_research_team existen en df_projects
valid_organization_ids = set(df_projects['project_id'])
df_research_team = df_research_team[df_research_team['project_id'].isin(valid_organization_ids)]

In [ ]:
df_research_team.head(2)

,project_id,nombre,apellido,email,user_id_type,user_dni,organization_id,job_position
5,PRDLR21531BRET,Jesús Manuel,Peregrina,jesusmanuel.peregrina@unirioja.es,DNI,16542732M,Universidad de La Rioja UniRIOJA,Thesis Director
13,TRNSC235653MACA,Tian,Tian,tiantian@vhio.net,NIE,Y2900276J,Vall dHebron Instituto de Oncologia VHIO,Team member


In [ ]:
df_research_team.dtypes

project_id         object
nombre             object
apellido           object
email              object
user_id_type       object
user_dni           object
organization_id    object
job_position       object
dtype: object

#### TABLA BUDGET

In [ ]:
def clean_and_adjust_dataframe_v4(file_path):
    # 1. Leer el archivo CSV en un DataFrame
    df_budget = leer_y_limpiar("Budget_Exported_AECC_v2.csv")
    # 2. Convertir las columnas de fecha a tipo datetime
    date_columns = ["starting_evaluation_period", "end_evaluation_period"]
    for col in date_columns:
        if col in df_budget.columns:
            df_budget[col] = pd.to_datetime(df_budget[col], format='%d/%m/%Y', errors="coerce")

    # 3. Identificar y convertir columnas al tipo adecuado (excepto las columnas de fecha)
    for col in df_budget.columns:
        if col not in date_columns:
            # Intentar convertir a float, si falla mantener como texto
            try:
                df_budget[col] = df_budget[col].astype(float)

                # Si se pudo convertir a float, intentar convertir a int (si no hay decimales)
                if df_budget[col].apply(float.is_integer).all():
                    df_budget[col] = df_budget[col].astype(int)
            except ValueError:
                continue

    # 4. Limpiar las columnas de texto
    string_columns = df_budget.select_dtypes(include=["object"]).columns
    for col in string_columns:
        df_budget[col] = df_budget[col].str.strip()

    return df_budget

# 1. Inicializamos y limpiamos el DataFrame
df_budget = clean_and_adjust_dataframe_v4("Budget_Exported_AECC_v2.csv")

# 2. Filtrar las filas de df_budget que tienen project_id en df_projects
df_budget = df_budget[df_budget['project_id'].isin(df_projects['project_id'])]


# 3. Asegúrate de que todos los project_id en df_budget existen en df_projects
valid_project_ids = set(df_projects['project_id'])
df_budget = df_budget[df_budget['project_id'].isin(valid_project_ids)]

In [ ]:
df_budget.head(2)

,project_id,budget_status,reporting_year,user_name,starting_evaluation_period,end_evaluation_period,budget_expendable_material_proposal,budget_equipment_proposal,budget_travel_expenses_proposal,budget_technical_services_proposal,budget_others_proposal,budget_total_amount_budget_proposal,total_budget_proposal,budget_expendable_material_current,budget_equipment_current,budget_travel_expenses_current,budget_technical_services_current,budget_others_current,budget_total_amount_current,total_budget_current,budget_expendable_material_justified,budget_equipment_justified,budget_travel_expenses_justified,budget_technical_services_justified,budget_others_justified,budget_total_amount_justified,total_budget_justified
0,AIO14142199GARC,Completed,Y4,Javier Garcia,2018-10-30,2019-07-31,22500.0,0.0,0.0,0.0,0.0,0.0,22500.0,22500.0,0.0,0.0,0.0,0.0,0.0,22500.0,30226.16,0.0,0.0,0.0,0.0,0.0,30226.16
1,AIO14142199GARC,Completed,Y5,Javier Garcia,2019-07-01,2020-07-22,22500.0,0.0,0.0,0.0,0.0,0.0,22500.0,22500.0,0.0,0.0,0.0,0.0,0.0,22500.0,49288.98,0.0,0.0,0.0,0.0,0.0,49288.98


In [ ]:
df_budget.dtypes

project_id                                      object
budget_status                                   object
reporting_year                                  object
user_name                                       object
starting_evaluation_period              datetime64[ns]
end_evaluation_period                   datetime64[ns]
budget_expendable_material_proposal            float64
budget_equipment_proposal                      float64
budget_travel_expenses_proposal                float64
budget_technical_services_proposal             float64
budget_others_proposal                         float64
budget_total_amount_budget_proposal            float64
total_budget_proposal                          float64
budget_expendable_material_current             float64
budget_equipment_current                       float64
budget_travel_expenses_current                 float64
budget_technical_services_current              float64
budget_others_current                          float64
budget_tot

#### TABLE USER

In [ ]:
# 1. Leer el archivo y vargando el df
df_user = leer_y_limpiar("Users_Exported_AECC_v2.csv")

# 2. Modificar fechas
df_user['date_birth'] = pd.to_datetime(df_user['date_birth'], errors='coerce')
df_user['user_update_date'] = pd.to_datetime(df_user['user_update_date'], format='%d/%m/%Y %H:%M').dt.normalize()

df_user['user_creation_date'] = df_user['user_creation_date'].replace('N/A', '2020-10-01')  # Truquito barato y sucio xd
df_user['user_creation_date'] = pd.to_datetime(df_user['user_creation_date']).dt.normalize()

# 3. Modificar Urls
df_user['user_orcid'] = df_user['user_orcid'].str.replace('https://orcid.org/', '', regex=False)
df_user['user_orcid'] = df_user['user_orcid'].str.replace('ORCID', '', regex=False)
df_user['user_orcid'] = df_user['user_orcid'].str.replace('orcid.org/', '', regex=False)
df_user['user_orcid'] = df_user['user_orcid'].str.replace('x', '', regex=False)
df_user['user_orcid'] = df_user['user_orcid'].str.replace('https://orcid.org/my-orcid?orcid=', '', regex=False)

# 4. Eliminar las filas con user_dni igual a '00000000X'
df_user = df_user[df_user['user_dni'] != '00000000X']

# 5. Haciendo magia
valid_project_ids = set(df_grant_type['grant_type'])
df_user.loc[~df_user['grant_type'].isin(valid_project_ids), 'grant_type'] = 'default'

# 6. Eliminar tildes y caracteres extraños de la columna organization_id
df_user['organization_id'] = df_user['organization_id'].apply(clean_organization_id)

# 7. Máscara booleana para identificar las filas que contienen "la Rioja"
mask = df_user['organization_id'].str.contains("la Rioja", case=False, na=False)
df_user.loc[mask, 'organization_id'] = df_user.loc[mask, 'organization_id'].str.replace("la Rioja", "La Rioja", case=False)

project_ids_validos = df_projects['project_id'].tolist()           # Obtén una lista de project_id válidos desde la tabla project
df_user = df_user[df_user['project_id'].isin(project_ids_validos)] # Filtra df_user para mantener solo las filas con project_id válido

In [ ]:
df_user.head(2)

,project_id,grant_type,user_name,user_short_name,user_surname,user_id_type,user_dni,date_birth,gender,place_birth,user_current_organization,organization_id,user_role,user_province,organization_province,user_city,user_country,user_cv_summary,user_orcid,user_creation_date,user_update_date
0,LABAE20001GONZ,LAB AECC 2020,Alicia Gonzalez,Alicia,Gonzalez Martin,DNI,14305313A,1980-03-10,Female,Madrid,Universidad Autónoma de Madrid (UAM),Universidad Autonoma de Madrid UAM,Researcher,Madrid,Madrid,Madrid,Spain,My laboratory focuses on identifying and study...,0000-0002-6179-089X,2020-10-01,2022-03-03
1,POSTD211561FUEN,Ayuda Postdoctoral AECC 2021,PEDRO FUENTES,PEDRO,FUENTES VARELA,DNI,47735882A,1988-02-25,Male,Barcelona,Instituto de Investigación Biomédica de Bellvi...,Instituto de Investigacion Biomedica de Bellvi...,Researcher,Barcelona,Barcelona,Barcelona,Spain,My first experience with biomedical research w...,0000-0001-8471-0197,2020-11-24,2022-03-14


In [ ]:
df_user.dtypes

project_id                           object
grant_type                           object
user_name                            object
user_short_name                      object
user_surname                         object
user_id_type                         object
user_dni                             object
date_birth                   datetime64[ns]
gender                               object
place_birth                          object
user_current_organization            object
organization_id                      object
user_role                            object
user_province                        object
organization_province                object
user_city                            object
user_country                         object
user_cv_summary                      object
user_orcid                           object
user_creation_date           datetime64[ns]
user_update_date             datetime64[ns]
dtype: object

In [ ]:
# @title TABLA AUX NEO4J: FUSION
# Podemos añadir más campos que consideremos necesarios
df_user_aux = pd.DataFrame()
df_user_aux = df_user[['project_id', 'user_dni', 'user_name']]
# ----------------
df_research_team_aux = pd.DataFrame()
df_research_team_aux = df_research_team[['project_id', 'user_dni']]
df_research_team_aux2 = df_research_team[['nombre', 'apellido']].copy()
df_research_team_aux2['user_name'] = df_research_team['nombre'] + ' ' + df_research_team['apellido']
df_research_team_aux = pd.concat([df_research_team_aux, df_research_team_aux2['user_name']], axis=1)
# ----------------
df_fusion = pd.concat([df_research_team_aux, df_user_aux], axis=0, ignore_index=True)
df_fusion = df_fusion.drop_duplicates()
# ----------------
df_fusion_pr = df_fusion['project_id']
df_fusion_pr = df_fusion_pr.drop_duplicates()
# ----------------
df_fusion_dni = df_fusion[['user_dni', 'user_name']]
df_fusion_dni = df_fusion_dni.drop_duplicates()

#### TABLA SCIENTIFIC_JUSTIFICATION



In [ ]:
# 1. Leer el archivo Scientific_Justifications_Exported_AECC_v1.xlsx
df_scientific_justifications = leer_y_limpiar("Scientific_Justifications_Exported_AECC_v1.csv")

# 2. Modificar fechas
df_scientific_justifications['deadline_scientific_just'] = pd.to_datetime(df_scientific_justifications['deadline_scientific_just'], errors='coerce')
df_scientific_justifications['starting_evaluation_period'] = pd.to_datetime(df_scientific_justifications['starting_evaluation_period'], errors='coerce')
df_scientific_justifications['end_evaluation_period'] = pd.to_datetime(df_scientific_justifications['end_evaluation_period'], errors='coerce')

# 3. Supongamos que df_scientific_justifications es tu DataFrame
mis_numeritos = ['additional_funding_impact_amount', 'people_hired', 'people_hired',
                   'women_predoctoral_hired', 'women_senior_posdoc_hired', 'women_technician_hired',
                   'people_project', 'project_executed']

# 4. Convertir las columnas a cadena (str) y llenar NaN con '0'
for col in mis_numeritos:
    df_scientific_justifications[col] = df_scientific_justifications[col].astype(str)
    df_scientific_justifications[col].fillna('0', inplace=True)

# 5. Luego, puedes convertir las columnas al tipo adecuado
df_scientific_justifications['additional_funding_impact_amount'] = pd.to_numeric(df_scientific_justifications['additional_funding_impact_amount'], errors='coerce')
df_scientific_justifications['people_hired'] = pd.to_numeric(df_scientific_justifications['people_hired'], errors='coerce')
df_scientific_justifications['women_posdoc_hired'] = pd.to_numeric(df_scientific_justifications['women_posdoc_hired'], errors='coerce')
df_scientific_justifications['women_predoctoral_hired'] = pd.to_numeric(df_scientific_justifications['women_predoctoral_hired'], errors='coerce')
df_scientific_justifications['women_senior_posdoc_hired'] = pd.to_numeric(df_scientific_justifications['women_senior_posdoc_hired'], errors='coerce')
df_scientific_justifications['women_technician_hired'] = pd.to_numeric(df_scientific_justifications['women_technician_hired'], errors='coerce')
df_scientific_justifications['people_project'] = pd.to_numeric(df_scientific_justifications['people_project'], errors='coerce')
df_scientific_justifications['project_executed'] = pd.to_numeric(df_scientific_justifications['project_executed'], errors='coerce')

df_scientific_justifications.replace('N/A', np.nan, inplace=True)

In [ ]:
df_scientific_justifications.head(2)

,project_id,description_scientific_just,type_scientific_just,status_scientific_just,deadline_scientific_just,starting_evaluation_period,end_evaluation_period,user_observation_scientific_just,scientific_just_evaluation_comments,scientific_desviation,patient_impact,patient_impact_description,additional_funding_impact,additional_funding_impact_origin,additional_funding_impact_grant,additional_funding_impact_amount,people_hired,women_posdoc_hired,women_predoctoral_hired,women_senior_posdoc_hired,women_technician_hired,people_project,clinical_impact,innovation_impact,project_executed,award_impact
0,PPLAB235418MART,Seguimiento Final,Final Report,Submitted,2023-10-15,2023-01-07,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PPLAB235304LORE,Seguimiento Final,Final Report,Completed,2023-09-15,2023-03-07,2023-08-31,"Tras hablar con Carmen Villana, me comentó la ...","Estimada Nerea,\nmuchas gracias por enviarnos ...",Sin desviación,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_scientific_justifications.dtypes

project_id                                     object
description_scientific_just                    object
type_scientific_just                           object
status_scientific_just                         object
deadline_scientific_just               datetime64[ns]
starting_evaluation_period             datetime64[ns]
end_evaluation_period                  datetime64[ns]
user_observation_scientific_just               object
scientific_just_evaluation_comments            object
scientific_desviation                          object
patient_impact                                 object
patient_impact_description                     object
additional_funding_impact                      object
additional_funding_impact_origin               object
additional_funding_impact_grant                object
additional_funding_impact_amount              float64
people_hired                                  float64
women_posdoc_hired                            float64
women_predoctoral_hired     

#### TABLA PROJECTS EXPORTED

In [ ]:
# 1. Leer el archivo Scientific_Justifications_Exported_AECC_v1.xlsx
df_pr_exported = leer_y_limpiar('Publications_Exported_AECC_v1.csv')

# 2. Cambios necesarios
df_pr_exported.replace('N/A', np.nan, inplace=True)
df_pr_exported['publication_type'] = df_pr_exported['publication_type'].replace('--Select--', 'NaN')
df_pr_exported['publication_status'] = df_pr_exported['publication_status'].replace('--Select--', 'NaN')
df_pr_exported['journal_impact_factor'] = df_pr_exported['journal_impact_factor'].str.replace(',', '.', regex=False)

# 3. Formatear la columna 'publication_date' al formato "YYYY-MM-DD"
df_pr_exported['publication_date'] = pd.to_datetime(df_pr_exported['publication_date'], errors="coerce")
df_pr_exported['publication_date'] = df_pr_exported['publication_date'].dt.strftime('%Y-%m-%d')

# 4. Expresión regular para extraer el número decimal
regex_pattern = r'^(\d+\.\d+)'
df_pr_exported['journal_impact_factor'] = df_pr_exported['journal_impact_factor'].str.extract(regex_pattern)

df_pr_exported['journal_impact_factor'] = df_pr_exported['journal_impact_factor'].fillna(0)

df_pr_exported['journal_quartile'] = df_pr_exported['journal_quartile'].fillna(0)
df_pr_exported['journal_quartile'] = df_pr_exported['journal_quartile'].str.replace(';', '')
df_pr_exported['journal_quartile'] = pd.to_numeric(df_pr_exported['journal_quartile'], errors='coerce').fillna(0)
df_pr_exported['journal_quartile'] = df_pr_exported['journal_quartile'].astype(int)

# 5. Tipos
df_pr_exported['journal_impact_factor'] = df_pr_exported['journal_impact_factor'].astype(float)
df_pr_exported['publication_date'] = pd.to_datetime(df_pr_exported['publication_date'], errors='coerce')
df_pr_exported['journal_quartile'] = df_pr_exported['journal_quartile'].astype(int)

# 6. Asegurarte de que todos los project_id en df_pr_exported existen en df_valid_projects
valid_project_ids = set(df_projects['project_id'])
df_pr_exported = df_pr_exported[df_pr_exported['project_id'].isin(valid_project_ids)]

In [ ]:
df_pr_exported.head(2)

In [ ]:
df_pr_exported.dtypes

project_id                             object
publication_title                      object
publication_abstract                   object
publication_first_author               object
publication_type                       object
journal                                object
publication_date               datetime64[ns]
publication_internal_status            object
journal_impact_factor                 float64
journal_quartile                        int64
publication_year                       object
publication_doi                        object
publication_status                     object
publication_pubmed                     object
dtype: object

# Guardar Datos en Drive

In [ ]:
df_user.to_csv(path + 'df_user.csv', index=False, encoding='utf-8')
df_budget.to_csv(path + 'df_budget.csv', index=False, encoding='utf-8')
df_grant_type.to_csv(path + 'df_grant_type.csv', index=False, encoding='utf-8')
df_invoice.to_csv(path + 'df_invoice.csv', index=False, encoding='utf-8')
df_organizations.to_csv(path + 'df_organizations.csv', index=False, encoding='utf-8')
df_projects.to_csv(path + 'df_projects.csv', index=False, encoding='utf-8')
df_research_types.to_csv(path + 'df_research_types.csv', index=False, encoding='utf-8')
df_cancer_types.to_csv(path + 'df_cancer_types.csv', index=False, encoding='utf-8')
df_cancer_phases.to_csv(path + 'df_cancer_phases.csv', index=False, encoding='utf-8')
df_scientific_justifications.to_csv(path + 'df_scientific_justifications.csv', index=False, encoding='utf-8')
df_research_team.to_csv(path + 'df_research_team.csv', index=False, encoding='utf-8')
df_project_research_type.to_csv(path + 'df_project_research_type.csv', index=False, encoding='utf-8')
df_project_cancer_type.to_csv(path + 'df_project_cancer_type.csv', index=False, encoding='utf-8')
df_project_cancer_phases.to_csv(path + 'df_project_cancer_phases.csv', index=False, encoding='utf-8')
df_pr_exported.to_csv(path + 'df_pr_exported.csv', index=False, encoding='utf-8')
df_fusion_ct_1.to_csv(path + 'df_fusion_ct_1.csv', index=False, encoding='utf-8')
df_fusion_ct_2.to_csv(path + 'df_fusion_ct_2.csv', index=False, encoding='utf-8')
df_fusion_rtc_3.to_csv(path + 'df_fusion_rtc_3.csv', index=False, encoding='utf-8')
df_fusion.to_csv(path + 'df_fusion.csv', index=False, encoding='utf-8')
df_fusion_pr.to_csv(path + 'df_fusion_pr.csv', index=False, encoding='utf-8')
df_fusion_dni.to_csv(path + 'df_fusion_dni.csv', index=False, encoding='utf-8')


In [ ]:
df_fusion_ct_1.to_csv(path + 'df_fusion_ct_1.csv', index=False, encoding='utf-8')
df_fusion_ct_2.to_csv(path + 'df_fusion_ct_2.csv', index=False, encoding='utf-8')
df_fusion_rtc_3.to_csv(path + 'df_fusion_rtc_3.csv', index=False, encoding='utf-8')
df_fusion.to_csv(path + 'df_fusion.csv', index=False, encoding='utf-8')
df_fusion_pr.to_csv(path + 'df_fusion_pr.csv', index=False, encoding='utf-8')
df_fusion_dni.to_csv(path + 'df_fusion_dni.csv', index=False, encoding='utf-8')

In [ ]:
dataframes = {
    'df_user': df_user,
    'df_budget': df_budget,
    'df_grant_type': df_grant_type,
    'df_invoice': df_invoice,
    'df_organizations': df_organizations,
    'df_projects': df_projects,
    'df_research_types': df_research_types,
    'df_cancer_types': df_cancer_types,
    'df_cancer_phases': df_cancer_phases,
    'df_scientific_justifications': df_scientific_justifications,
    'df_research_team': df_research_team,
    'df_project_research_type': df_project_research_type,
    'df_project_cancer_type': df_project_cancer_type,
    'df_project_cancer_phases': df_project_cancer_phases,
    'df_pr_exported': df_pr_exported,
    'df_fusion_ct_1': df_fusion_ct_1,
    'df_fusion_ct_2': df_fusion_ct_2,
    'df_fusion_rtc_3': df_fusion_rtc_3,
    'df_fusion': df_fusion,
    'df_fusion_pr': df_fusion_pr,
    'df_fusion_dni': df_fusion_dni
}

for name, df in dataframes.items():
    df.to_csv(f"{path}{name}.csv", index=False, encoding='utf-8')


In [ ]:
# @title CARGA DE DATOS A LA MV-GCP
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from pandas import Series
import string
import re
import numpy as np


!pip install psycopg2-binary sqlalchemy
from sqlalchemy import create_engine
#import psycopg2



local_path = '/content/drive/Shared drives/TFM/BBDD/EXPORTED/datos_exported/'

df_user = pd.read_csv(local_path + 'df_user.csv', encoding='utf-8')
df_budget = pd.read_csv(local_path + 'df_budget.csv', encoding='utf-8')
df_grant_type = pd.read_csv(local_path + 'df_grant_type.csv', encoding='utf-8')
df_invoice = pd.read_csv(local_path + 'df_invoice.csv', encoding='utf-8')
df_organizations = pd.read_csv(local_path + 'df_organizations.csv', encoding='utf-8')
df_projects = pd.read_csv(local_path + 'df_projects.csv', encoding='utf-8')
df_research_types = pd.read_csv(local_path + 'df_research_types.csv', encoding='utf-8')
df_cancer_types = pd.read_csv(local_path + 'df_cancer_types.csv', encoding='utf-8')
df_cancer_phases = pd.read_csv(local_path + 'df_cancer_phases.csv', encoding='utf-8')
df_scientific_justifications = pd.read_csv(local_path + 'df_scientific_justifications.csv', encoding='utf-8')
df_research_team = pd.read_csv(local_path + 'df_research_team.csv', encoding='utf-8')
df_project_research_type = pd.read_csv(local_path + 'df_project_research_type.csv', encoding='utf-8')
df_project_cancer_type = pd.read_csv(local_path + 'df_project_cancer_type.csv', encoding='utf-8')
df_project_cancer_phases = pd.read_csv(local_path + 'df_project_cancer_phases.csv', encoding='utf-8')
df_pr_exported = pd.read_csv(local_path + 'df_pr_exported.csv', encoding='utf-8')


# Parámetros de conexión
host = "{ipgcp}"
port = "5432"
database = "postgres"
user = "postgres"
password = "{password}"

# Construye la cadena de conexión DATABASE_URL
DATABASE_URL = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"

# Crea el motor de conexión
engine = create_engine(DATABASE_URL)

if_exists_value = "append"

########### REPLACE: AÑADIR TABLAS y SUSTITUIR VALORES ################

########## APPEND: AÑADIR REGISTROS NUEVOS A UNA TABLA EXISTENTE ####

df_grant_type.to_sql('grant_types', engine, if_exists=if_exists_value, index=False)
df_organizations.to_sql('organizations', engine, if_exists=if_exists_value, index=False)
df_projects.to_sql('projects', engine, if_exists=if_exists_value, index=False)
df_cancer_phases.to_sql('cancer_phases', engine, if_exists=if_exists_value, index=False)
df_user.to_sql('users', engine, if_exists=if_exists_value, index=False)
df_pr_exported.to_sql('publications', engine, if_exists=if_exists_value, index=False)
df_budget.to_sql('budget', engine, if_exists=if_exists_value, index=False)
df_invoice.to_sql('invoice', engine, if_exists=if_exists_value, index=False)
df_research_types.to_sql('research_types', engine, if_exists=if_exists_value, index=False)
df_cancer_types.to_sql('cancer_type', engine, if_exists=if_exists_value, index=False)
df_scientific_justifications.to_sql('scientific_justifications', engine, if_exists=if_exists_value, index=False)
df_research_team.to_sql('research_team', engine, if_exists=if_exists_value, index=False)
df_project_research_type.to_sql('project_research_type', engine, if_exists=if_exists_value, index=False)
df_project_cancer_type.to_sql('project_cancer_type', engine, if_exists=if_exists_value, index=False)
df_project_cancer_phases.to_sql('project_cancer_phases', engine, if_exists=if_exists_value, index=False)
# para añadir registros:
# users_df.to_sql('users', engine, if_exists='append', index=False)